In [1]:
import pickle
import networkx as nx

G = pickle.load(open("./output/thirsty-crow-prompt2_nx_graph.pkl", "rb"))
G

In [ ]:
import networkx as nx
import ollama
import json
from itertools import combinations


with open("./input/config.json", "r") as file:
    config = json.load(file)

model = config['model_name']

system_prompt_root_word = """You are a helpful assistant that extracts the root form of the given word in lowercase.
Return only the root word in response."""

system_prompt_key_words = """You are a helpful assistant that extracts keywords from the given question in lowercase.
The returned keywords should be critical to answer the question.
The returned words should be separated by space.
Return only the keywords in response separated by ','."""

query = "how was the crow able to drink water?"

def get_llm_response(text, system_prompt):
    response = ollama.chat(model=model, messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": text}
        ])
    # print(f'response={response['message']['content']}')
    resp_content = response['message']['content']
    return resp_content

# --- Step 2: Retrieve relevant triples ---
def search_kg(G, query):
    print(f"no. of edges in KG: {G.number_of_edges()}")
    results = []
    for u, v, d in G.edges(data=True):
        relation = d.get('label', None) if d else None
        relation = d.get('relation', None) if relation is None else relation
        triple = (u, relation, v)
        print(f'triple: {triple}')
        # naive: check if query words appear in triple

        if any(get_llm_response(word, system_prompt_root_word) in " ".join(map(str, triple)).lower() for word in query.split()):
            results.append(triple)
    return results

def search_kg2(G, query):
    keywords = get_llm_response(query, system_prompt_key_words)
    print(f'keywords={keywords}')
    keywords = keywords.split(',')

    keywords = ['crow', 'water', 'drop']
    pairs = list(combinations(keywords, 2))
    print(f'pairs={pairs}')
    for u, v in pairs:
        paths = list(nx.all_simple_paths(G, source=u, target=v))
        print(f'paths={paths}')
        for path in paths:
            relations = []
            for i in range(len(path)-1):
                print(f'path{i}[{path[i]},{path[i+1]}]=>{G[path[i]]}-->{G[path[i]][path[i+1]]}')
                for key in G[path[i]][path[i+1]]:
                    print(f'key={key}, value={G[path[i]][path[i+1]][key]}')
                    rel = G[path[i]][path[i+1]][key]['relation']
                relations.append((path[i],rel, path[i+1]))

    print(f'relations={relations}')
    return relations

triples = search_kg2(G, query)
print(f'number of relevant triples found: {len(triples)}, {triples}')

# --- Step 3: Send to Ollama model ---
context = f"""
You are given facts from a knowledge graph:

{triples}

Answer the user query based ONLY on these facts.
Answer in full sentence.
Query: {query}
"""

response = ollama.chat(model="gemma3:4b-it-qat", messages=[{"role": "user", "content": context}])

print(response["message"]["content"])


keywords=crow, water, drink, able
pairs=[('crow', 'water'), ('crow', 'drop'), ('water', 'drop')]
paths=[['crow', 'water'], ['crow', 'water'], ['crow', 'water'], ['crow', 'water'], ['crow', 'water'], ['crow', 'pitcher', 'water'], ['crow', 'stone', 'pitcher', 'water']]
path0[crow,water]=>{'water': {0: {'relation': 'fly_in_search_of'}, 1: {'relation': 'see'}, 2: {'relation': 'try_to_reach'}, 3: {'relation': 'drink'}, 4: {'relation': 'fly_away'}}, 'pitcher': {0: {'relation': 'look_inside'}}, 'stone': {0: {'relation': 'drop_in'}}}-->{0: {'relation': 'fly_in_search_of'}, 1: {'relation': 'see'}, 2: {'relation': 'try_to_reach'}, 3: {'relation': 'drink'}, 4: {'relation': 'fly_away'}}
key=0, value={'relation': 'fly_in_search_of'}
key=1, value={'relation': 'see'}
key=2, value={'relation': 'try_to_reach'}
key=3, value={'relation': 'drink'}
key=4, value={'relation': 'fly_away'}
path0[crow,water]=>{'water': {0: {'relation': 'fly_in_search_of'}, 1: {'relation': 'see'}, 2: {'relation': 'try_to_reach'}

paths=[['crow', 'water'], ['crow', 'water'], ['crow', 'water'], ['crow', 'water'], ['crow', 'water'], ['crow', 'pitcher', 'water'], ['crow', 'stone', 'pitcher', 'water']]
path0[crow,water]=>{'water': {0: {'relation': 'fly_in_search_of'}, 1: {'relation': 'see'}, 2: {'relation': 'try_to_reach'}, 3: {'relation': 'drink'}, 4: {'relation': 'fly_away'}}, 'pitcher': {0: {'relation': 'look_inside'}}, 'stone': {0: {'relation': 'drop_in'}}}-->{0: {'relation': 'fly_in_search_of'}, 1: {'relation': 'see'}, 2: {'relation': 'try_to_reach'}, 3: {'relation': 'drink'}, 4: {'relation': 'fly_away'}}
key=0, value={'relation': 'fly_in_search_of'}
key=1, value={'relation': 'see'}
key=2, value={'relation': 'try_to_reach'}
key=3, value={'relation': 'drink'}
key=4, value={'relation': 'fly_away'}
path0[crow,water]=>{'water': {0: {'relation': 'fly_in_search_of'}, 1: {'relation': 'see'}, 2: {'relation': 'try_to_reach'}, 3: {'relation': 'drink'}, 4: {'relation': 'fly_away'}}, 'pitcher': {0: {'relation': 'look_insid